Spam Detection


In [1]:
import tensorflow as tf
import csv
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Data Preprocessing

In [2]:
all_sentences = []
all_labels = []
count = 0
dataset = pd.read_csv('spam.csv', encoding='latin-1')
all_data = dataset.iloc[:, :2]
all_data.rename(columns={'v1':'result', 'v2':'text'}, inplace=True)
all_data.sample(10)

,result,text
3194,ham,Great. P diddy is my neighbor and comes for to...
72,ham,HI BABE IM AT HOME NOW WANNA DO SOMETHING? XX
5351,ham,Yup it's at paragon... I havent decided whethe...
4612,ham,How much you got for cleaning
3038,ham,"Wishing you and your family Merry \X\"" mas and..."
1333,ham,"Oh... Icic... K lor, den meet other day..."
5247,ham,Gud ni8 dear..slp well..take care..swt dreams....
191,ham,I'm sorry. I've joined the league of people th...
3258,spam,Congratulations! Thanks to a good friend U hav...
2699,ham,Oh baby of the house. How come you dont have a...


Encoding the dependent variable


In [3]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

#spam = 1, not spam = 0
all_data['result'] = encoder.fit_transform(all_data['result'])
print(all_data)

      result                                               text
0          0  Go until jurong point, crazy.. Available only ...
1          0                      Ok lar... Joking wif u oni...
2          1  Free entry in 2 a wkly comp to win FA Cup fina...
3          0  U dun say so early hor... U c already then say...
4          0  Nah I don't think he goes to usf, he lives aro...
...      ...                                                ...
5567       1  This is the 2nd time we have tried 2 contact u...
5568       0              Will Ì_ b going to esplanade fr home?
5569       0  Pity, * was in mood for that. So...any other s...
5570       0  The guy did some bitching but I acted like i'd...
5571       0                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [4]:
all_data.isnull().sum()

result    0
text      0
dtype: int64

In [5]:
all_data = all_data.drop_duplicates(keep='first')

In [6]:
all_data.duplicated().sum()


0

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
all_data['num_words'] = all_data['text'].apply(lambda x: len(nltk.word_tokenize(x)))
all_data['num_sentences'] = all_data['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [9]:
all_data.sample(10)

,result,text,num_words,num_sentences
132,0,First answer my question.,5,1
3505,0,Nite...,2,1
4251,0,"Send ur birthdate with month and year, I will ...",27,3
4089,1,We tried to call you re your reply to our sms ...,27,1
5210,0,Dai i downloaded but there is only exe file wh...,19,1
3965,0,I was just callin to say hi. Take care bruv!,12,2
3969,0,That's the trouble with classes that go well -...,31,2
2538,0,The monthly amount is not that terrible and yo...,19,1
573,0,Waiting for your call.,5,1
156,0,I'm leaving my house now...,7,1


Getting rid of stopwords and tokenizing

In [10]:
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
def data_preprocessing(text):
  final_format = []
  text = text.lower()
  text = word_tokenize(text)
  for word in text:
    if word not in stopwords.words('english') and word not in string.punctuation:
      final_format.append(word)
  return " ".join(final_format)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
all_data['converted_text'] = all_data['text'].apply(data_preprocessing)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)
x = tfidf.fit_transform(all_data['converted_text']).toarray()
##x = all_data.iloc[:, 4:5].values
y = all_data.iloc[:, 0:1].values
x.shape


(5169, 3000)

Splitting the data into test and train set


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)
print(x_train)
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


Prediction and Accuracy Result

In [14]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()


bnb.fit(x_train,y_train)
y_pred3 = bnb.predict(x_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9864603481624759
[[896   0]
 [ 14 124]]
1.0
